In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#variables
url='/content/drive/MyDrive/dataRev2/'

In [ ]:
author=pd.read_csv(url+'Author.csv')
author.drop_duplicates(keep='first',inplace=True,ignore_index=True)
author.head()

,Id,Name,Affiliation
0,9,Ernest Jordan,NaN
1,14,K. MORIBE,NaN
2,15,D. Jakominich,NaN
3,25,William H. Nailon,NaN
4,37,P. B. Littlewood,Cavendish Laboratory|Cambridge University


In [ ]:
conference=pd.read_csv(url+'Conference.csv')
conference.drop_duplicates(keep='first',inplace=True,ignore_index=True)
conference.head()

,Id,ShortName,FullName,HomePage
0,1,IADIS,International Association for Development of t...,NaN
1,2,IADT,Issues and Applications of Database Technology,http://www.informatik.uni-trier.de/~ley/db/con...
2,4,NaN,IBM Germany Scientific Symposium Series,http://www.informatik.uni-trier.de/~ley/db/con...
3,5,ICOMP,International Conference on Internet Computing,http://www.informatik.uni-trier.de/~ley/db/con...
4,6,ICAC,International Conference on Autonomic Computing,http://www.autonomic-conference.org/


In [ ]:
journal=pd.read_csv(url+'Journal.csv')
journal.drop_duplicates(keep='first',inplace=True,ignore_index=True)
journal.head()

,Id,ShortName,FullName,HomePage
0,1,ICOM,Zeitschrift Für Interaktive Und Kooperative Me...,http://www.i-com-media.de
1,2,AEPIA,"Inteligencia Artificial,revista Iberoamericana...",http://aepia.dsic.upv.es/revista/
2,3,IBMRD,Ibm Journal of Research and Development,http://www-tr.watson.ibm.com/journal/rdindex.html
3,4,IBMSJ,Ibm Systems Journal,http://researchweb.watson.ibm.com/journal/
4,5,NaN,Iet Software/iee Proceedings - Software,http://www.ietdl.org/IET-SEN


In [ ]:
paper=pd.read_csv(url+'Paper.csv')
paper.drop_duplicates(keep='first',inplace=True,ignore_index=True)
paper.head()

,Id,Title,Year,ConferenceId,JournalId,Keyword
0,1,Stitching videos streamed by mobile phones in ...,2009,167,0,mobile video capturing|real-time|video stitching
1,2,A nonlocal convection–diffusion equation,2007,0,7234,Nonlocal diffusion; Convection–diffusion; Asym...
2,3,Area Effects in Cepaea,1963,0,16867,NaN
3,4,Multiple paternity in a natural population of ...,2005,0,6130,NaN
4,5,Complexity of Finding Short Resolution Proofs,1997,158,0,NaN


In [ ]:
paperauthor=pd.read_csv(url+'PaperAuthor.csv')
paperauthor.drop_duplicates(keep='first',inplace=True,ignore_index=True)
paperauthor.head()

,PaperId,AuthorId,Name,Affiliation
0,1,521630,Ayman Kaheel,Cairo Microsoft Innovation Lab
1,1,972575,Mahmoud Refaat,Cairo Microsoft Innovation Lab
2,1,1528710,Ahmed Abdul-hamid,Cairo Microsoft Innovation Lab
3,1,1611750,Motaz Ahmad El-saban,Cairo Microsoft Innovation Lab
4,2,1682088,Julio D. Rossi,"Departamento Matemática, FCEyN UBA (1428) Buen..."


In [ ]:
train=pd.read_csv(url+'Train.csv')
train.drop_duplicates(keep='first',inplace=True,ignore_index=True)
train.head()

,AuthorId,ConfirmedPaperIds,DeletedPaperIds
0,826,25733 47276 77012 79468 87141 101385 104556 11...,220293 379873 384468 565252 939749 944658 1164...
1,933,1739240,477879
2,1118,49963 93433 341015 415282 488635 517119 521922...,961444 1446566
3,2783,154377 212636 334024 350747 696269 704564 1241...,208521 1207662 1898210
4,3105,603562 647574 792910 844605 863071 878798 1082...,191079 226980 273592 529834 556687 1052455 131...


In [ ]:
test=pd.read_csv(url+'Valid.csv')
test.drop_duplicates(keep='first',inplace=True,ignore_index=True)
test.head()

,AuthorId,PaperIds
0,55,2507 15471 19294 20444 24074 25103 25304 36459...
1,426,873 37257 92584 174625 212235 284639 581263 60...
2,1861,59550 124595 282043 528639 922192 1048827 1059...
3,10212,24013 49767 131299 173546 195044 199989 213826...
4,10275,300441 358576 368138 389312 871240 876311 9952...


Paperauthor is the source dataset so i am going to join the rest of tables to it :p 

In [ ]:
#add the paper dataset
full_dataset=paperauthor.merge(paper,left_on='PaperId',right_on='Id',how='left')
full_dataset.drop('Id',axis=1,inplace=True)
#add the conference dataset
full_dataset=full_dataset.merge(conference,left_on='ConferenceId',right_on='Id',how='left')
full_dataset.drop('Id',axis=1,inplace=True)
full_dataset=full_dataset.rename(columns={'FullName':'ConferenceName','ShortName':'ConferenceShortName','HomePage':'ConferenceHomePage'})
#add the journal dataset
full_dataset=full_dataset.merge(journal,left_on='JournalId',right_on='Id',how='left')
full_dataset=full_dataset.rename(columns={'FullName':'JournalName','ShortName':'JournalShortName','HomePage':'JournalHomePage'})
full_dataset.drop('Id',axis=1,inplace=True)

full_dataset.head()


,PaperId,AuthorId,Name,Affiliation,Title,Year,ConferenceId,JournalId,Keyword,ConferenceShortName,ConferenceName,ConferenceHomePage,JournalShortName,JournalName,JournalHomePage
0,1,521630,Ayman Kaheel,Cairo Microsoft Innovation Lab,Stitching videos streamed by mobile phones in ...,2009.0,167.0,0.0,mobile video capturing|real-time|video stitching,MM,ACM Multimedia Conference,http://acmmm2011.utdallas.edu/page14.html,NaN,NaN,NaN
1,1,972575,Mahmoud Refaat,Cairo Microsoft Innovation Lab,Stitching videos streamed by mobile phones in ...,2009.0,167.0,0.0,mobile video capturing|real-time|video stitching,MM,ACM Multimedia Conference,http://acmmm2011.utdallas.edu/page14.html,NaN,NaN,NaN
2,1,1528710,Ahmed Abdul-hamid,Cairo Microsoft Innovation Lab,Stitching videos streamed by mobile phones in ...,2009.0,167.0,0.0,mobile video capturing|real-time|video stitching,MM,ACM Multimedia Conference,http://acmmm2011.utdallas.edu/page14.html,NaN,NaN,NaN
3,1,1611750,Motaz Ahmad El-saban,Cairo Microsoft Innovation Lab,Stitching videos streamed by mobile phones in ...,2009.0,167.0,0.0,mobile video capturing|real-time|video stitching,MM,ACM Multimedia Conference,http://acmmm2011.utdallas.edu/page14.html,NaN,NaN,NaN
4,2,1682088,Julio D. Rossi,"Departamento Matemática, FCEyN UBA (1428) Buen...",A nonlocal convection–diffusion equation,2007.0,0.0,7234.0,Nonlocal diffusion; Convection–diffusion; Asym...,NaN,NaN,NaN,J FUNCT ANAL,Journal of Functional Analysis,http://www.sciencedirect.com/science/journal/0...


In [ ]:
full_dataset.drop('Name',axis=1,inplace=True)

In [ ]:
full_dataset.Affiliation=[str(x) for x in full_dataset.Affiliation]
full_dataset.Keyword=[str(x) for x in full_dataset.Keyword]
full_dataset.JournalName=[str(x) for x in full_dataset.JournalName]
full_dataset.ConferenceShortName=[str(x) for x in full_dataset.ConferenceShortName]
full_dataset.JournalShortName=[str(x) for x in full_dataset.JournalShortName]
full_dataset.ConferenceName=[str(x) for x in full_dataset.ConferenceName]

In [ ]:
le=LabelEncoder()
le.fit(np.array(full_dataset.Affiliation))
full_dataset.Affiliation=le.transform(full_dataset.Affiliation)

In [ ]:
le=LabelEncoder()
le.fit(np.array(full_dataset.Keyword))
full_dataset.Keyword=le.transform(full_dataset.Keyword)


In [ ]:
le=LabelEncoder()
le.fit(np.array(full_dataset.ConferenceShortName))
full_dataset.ConferenceShortName=le.transform(full_dataset.ConferenceShortName)

In [ ]:
le=LabelEncoder()
le.fit(np.array(full_dataset.ConferenceShortName))
full_dataset.ConferenceShortName=le.transform(full_dataset.ConferenceShortName)

In [ ]:
le=LabelEncoder()
le.fit(np.array(full_dataset.ConferenceName))
full_dataset.ConferenceName=le.transform(full_dataset.ConferenceName)

In [ ]:
full_dataset.drop('ConferenceHomePage',axis=True,inplace=True)

In [ ]:
le=LabelEncoder()
le.fit(np.array(full_dataset.JournalShortName))
full_dataset.JournalShortName=le.transform(full_dataset.JournalShortName)

In [ ]:
le=LabelEncoder()
le.fit(np.array(full_dataset.JournalName))
full_dataset.JournalName=le.transform(full_dataset.JournalName)

In [ ]:
full_dataset.drop('JournalHomePage',axis=1,inplace=True)

In [ ]:
full_dataset.drop('Title',axis=1,inplace=True)

In [ ]:
full_dataset.head()

,PaperId,AuthorId,Affiliation,Year,ConferenceId,JournalId,Keyword,ConferenceShortName,ConferenceName,JournalShortName,JournalName
0,1,521630,54309,2009.0,167.0,0.0,468751,2478,35,8283,15106
1,1,972575,54309,2009.0,167.0,0.0,468751,2478,35,8283,15106
2,1,1528710,54309,2009.0,167.0,0.0,468751,2478,35,8283,15106
3,1,1611750,54309,2009.0,167.0,0.0,468751,2478,35,8283,15106
4,2,1682088,123799,2007.0,0.0,7234.0,339185,3691,4473,4682,8398


In [ ]:
full_train=full_dataset.loc[full_dataset.AuthorId.isin(train.AuthorId),:]
full_train.index=range(full_train.shape[0])
full_train.head()

,PaperId,AuthorId,Affiliation,Year,ConferenceId,JournalId,Keyword,ConferenceShortName,ConferenceName,JournalShortName,JournalName
0,8,318484,718871,1985.0,0.0,4084.0,470435,3691,4473,6669,11776
1,9,2151636,570520,2005.0,0.0,3943.0,293448,3691,4473,8283,11081
2,10,1925401,557636,0.0,0.0,0.0,470435,3691,4473,8283,15106
3,12,2074708,789791,2003.0,1231.0,0.0,470435,2047,1190,8283,15106
4,12,2074708,250331,2003.0,1231.0,0.0,470435,2047,1190,8283,15106


In [ ]:
confirmed=[]
for i in range(full_train.shape[0]):
  if str(full_train.PaperId[i]) in str(train.loc[train.AuthorId==full_train.AuthorId[i],'ConfirmedPaperIds']).split():
    confirmed.append(1)
  else:
    confirmed.append(0)
    

In [ ]:
full_train=pd.concat([pd.Series(confirmed,name='confirmed'),full_train],axis=1)
full_train.head()

,confirmed,PaperId,AuthorId,Affiliation,Year,ConferenceId,JournalId,Keyword,ConferenceShortName,ConferenceName,JournalShortName,JournalName
0,0,8,318484,718871,1985.0,0.0,4084.0,470435,3691,4473,6669,11776
1,0,9,2151636,570520,2005.0,0.0,3943.0,293448,3691,4473,8283,11081
2,0,10,1925401,557636,0.0,0.0,0.0,470435,3691,4473,8283,15106
3,1,12,2074708,789791,2003.0,1231.0,0.0,470435,2047,1190,8283,15106
4,1,12,2074708,250331,2003.0,1231.0,0.0,470435,2047,1190,8283,15106


In [ ]:
full_train.dropna(inplace=True)

In [ ]:
x=full_train.drop('confirmed',axis=1).values
y=full_train.confirmed.values

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.5)

In [ ]:
full_test=full_dataset.loc[full_dataset.AuthorId.isin(test.AuthorId),:]
full_test.index=range(full_test.shape[0])
full_test.dropna(inplace=True)
full_test.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PaperId,AuthorId,Affiliation,Year,ConferenceId,JournalId,Keyword,ConferenceShortName,ConferenceName,JournalShortName,JournalName
0,37,1303266,671475,2010.0,0.0,2399.0,470435,3691,4473,43,148
1,53,1037194,623060,2009.0,2058.0,0.0,248136,1581,2366,8283,15106
2,53,1037194,653994,2009.0,2058.0,0.0,248136,1581,2366,8283,15106
3,60,2002249,828286,2011.0,0.0,4921.0,55766,3691,4473,5324,9382
4,60,2002249,538079,2011.0,0.0,4921.0,55766,3691,4473,5324,9382


In [ ]:
del full_dataset,train,paper,author,paperauthor,journal,conference

In [ ]:
model1=RandomForestClassifier()
model2=LogisticRegression()

model1.fit(x_train,y_train)
model2.fit(x_train,y_train)

pred1=model1.predict(x_test)
pred2=model2.predict(x_test)

test_pred1=model1.predict(full_test)
test_pred2=model2.predict(full_test)

stack_prediction=np.column_stack((pred1,pred2))
stack_test_prediction=np.column_stack((test_pred1,test_pred2))

meta_model=SVC()
meta_model.fit(stack_prediction,y_test)

final_prediction=meta_model.predict(stack_test_prediction)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
new_test=pd.concat([full_test.AuthorId,full_test.PaperId,pd.Series(final_prediction,name='confirmed')],axis=1)
new_test.fillna(0,inplace=True)
new_test.head()

,AuthorId,PaperId,confirmed
0,1303266.0,37.0,1.0
1,1037194.0,53.0,1.0
2,1037194.0,53.0,1.0
3,2002249.0,60.0,1.0
4,2002249.0,60.0,0.0


In [ ]:
sorted_test=new_test.sort_values(by=['AuthorId'])
sorted_test.head()

,AuthorId,PaperId,confirmed
104789,0.0,0.0,0.0
121785,0.0,0.0,0.0
136748,0.0,0.0,0.0
66684,0.0,0.0,0.0
2088,0.0,0.0,0.0


In [ ]:
PaperId=[]
for i in sorted_test.AuthorId.unique():
  string=""
  for j in sorted_test.loc[sorted_test.AuthorId==i,'PaperId']:
    string+=string+" "+str(j)
  PaperId.append(string)

In [ ]:
PaperID=pd.Series(PaperId)